In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(25)
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator


Loading data


In [ ]:
mnist_dataset = fetch_openml('mnist_784')
print(type(mnist_dataset))

<class 'sklearn.utils.Bunch'>


In [ ]:
X = mnist_dataset['data']
print('X', type(X), X.shape)

X <class 'numpy.ndarray'> (70000, 784)


In [ ]:
y = mnist_dataset['target']
print('y', type(y), y.shape)
print('split to train', y[:59999])
print('split to test', y[60000:])

y <class 'numpy.ndarray'> (70000,)
split to train ['5' '0' '4' ... '3' '5' '6']
split to test ['7' '2' '1' ... '4' '5' '6']


In [ ]:
X_train, X_test = X[:60000], X[60000:]
print('X_train', type(X_train), X_train.shape)
print('X_test', type(X_test), X_test.shape)

X_train <class 'numpy.ndarray'> (60000, 784)
X_test <class 'numpy.ndarray'> (10000, 784)


In [ ]:
y_train, y_test = y[:60000], y[60000:]
print('y_train', type(y_train), y_train.shape)
print('y_test', type(y_test), y_test.shape)

y_train <class 'numpy.ndarray'> (60000,)
y_test <class 'numpy.ndarray'> (10000,)


Preprocessing

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train/=255
X_test/=255

X_train.shape

(60000, 28, 28, 1)

In [ ]:
from keras.utils import np_utils
number_of_classes = 10

Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)

y_train[0], Y_train[0]

('5', array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32))

CNN Model


In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

Training Model

In [ ]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
epochs = 1 # Turn epochs to 30 to get 0.9967 accuracy
batch_size = 86


# With data augmentation to prevent overfitting (accuracy 0.99286)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)



In [ ]:

model.fit_generator(train_generator, steps_per_epoch=60000//64, epochs=5, 
                    validation_data=test_generator, validation_steps=10000//64)

Epoch 1/5
937/937 [==============================] - 426s 455ms/step - loss: 0.0671 - accuracy: 0.9829 - val_loss: 2.3031 - val_accuracy: 0.1132
Epoch 2/5
937/937 [==============================] - 428s 457ms/step - loss: 0.0564 - accuracy: 0.9837 - val_loss: 2.3033 - val_accuracy: 0.1136
Epoch 3/5
937/937 [==============================] - 428s 457ms/step - loss: 0.0578 - accuracy: 0.9839 - val_loss: 2.3038 - val_accuracy: 0.1136
Epoch 4/5
937/937 [==============================] - 434s 463ms/step - loss: 0.0575 - accuracy: 0.9843 - val_loss: 2.3040 - val_accuracy: 0.1134
Epoch 5/5
937/937 [==============================] - 426s 455ms/step - loss: 0.0632 - accuracy: 0.9830 - val_loss: 2.3044 - val_accuracy: 0.1136


Test Accuracy

In [ ]:
score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])

313/313 [==============================] - 16s 52ms/step - loss: 0.2711 - accuracy: 0.9298

Test accuracy:  0.9297999739646912
